In [1]:
import boto3

# import pandas and assign to the popular 'pd' convention
import pandas as pd 

In [2]:
# Setup a region
region = 'us-west-2'
# Create an S3 client
s3 = boto3.client('s3',region_name = region)

# Creates a variable with the bucket name
bucket_name = 'unknown-tm-analysis-20200302' #'<insert a unique bucket name>' 

# Create a location Constraint
location = {'LocationConstraint': region}
# Creates a new bucket 
s3.create_bucket(Bucket=bucket_name,CreateBucketConfiguration=location)

{'ResponseMetadata': {'RequestId': '335FE0842237CD71',
  'HostId': 'A80wI6gGBaQrUWuk4tSUMJYOep7ZAcIuD6myH2+em5p+ClqI4Qm1GMqGMZvTAc9PvejnR6eiivg=',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': 'A80wI6gGBaQrUWuk4tSUMJYOep7ZAcIuD6myH2+em5p+ClqI4Qm1GMqGMZvTAc9PvejnR6eiivg=',
   'x-amz-request-id': '335FE0842237CD71',
   'date': 'Tue, 03 Mar 2020 03:26:03 GMT',
   'location': 'http://unknown-tm-analysis-20200302.s3.amazonaws.com/',
   'content-length': '0',
   'server': 'AmazonS3'},
  'RetryAttempts': 0},
 'Location': 'http://unknown-tm-analysis-20200302.s3.amazonaws.com/'}

In [3]:
# Create a list of file names
filenames_list = ['doc-topics.csv', 'topic-terms.csv']

# Iterates on each file in the  filenames_list
for filename in filenames_list:
    # Uploads each CSV to the created bucket
    s3.upload_file(filename, bucket_name, filename)
    # checks if the filename is 'doc-topics.csv'
    if filename == 'doc-topics.csv':
        # gets the 'doc-topics.csv' file as an object
        obj = s3.get_object(Bucket=bucket_name, Key=filename)
        # reads the csv and assigns to doc_topics 
        doc_topics = pd.read_csv(obj['Body'])
    else:
        obj = s3.get_object(Bucket=bucket_name, Key=filename)
        topic_terms = pd.read_csv(obj['Body'])

# merge files on topic column to obtain the most common terms per document
merged_df = pd.merge(doc_topics, topic_terms, on='topic')

# print the merged_df to the console
print(merged_df) 

              docname  topic  proportion      term    weight
0      cv125_9636.txt      5    0.791506      time  0.010082
1      cv125_9636.txt      5    0.791506     jones  0.005874
2      cv125_9636.txt      5    0.791506  fugitive  0.003999
3      cv125_9636.txt      5    0.791506       lee  0.004330
4      cv125_9636.txt      5    0.791506      plot  0.007428
...               ...    ...         ...       ...       ...
43985  cv533_9843.txt     36    0.094384      hack  0.013879
43986  cv533_9843.txt     36    0.094384     young  0.013752
43987  cv533_9843.txt     36    0.094384     movie  0.029418
43988  cv533_9843.txt     36    0.094384    murphy  0.010977
43989  cv533_9843.txt     36    0.094384      time  0.015294

[43990 rows x 5 columns]
